## Journal

### Extracting nodules and lables 

For all 9 features of the annotations

The Lung Image Database Consortium image collection (LIDC-IDRI) consists of diagnostic and lung cancer screening thoracic computed tomography (CT) scans with marked-up annotated lesions. This database consists of 3-D CT scans along with an xml file for each patient. This xml files contains unblinded and blinded markings from 4 different radiologists. The masses detected are categorized into three different types.

- Non-nodules
- Nodules with diameters <3mm
- Nodules with diameter > 3mm

Each of these have a x,y,z demarcation of their centers. But only the nodules greater than 3mm diameter are considered for this study. These  xmls contains 9 different characteristics for nodules with diameter >3mm namely Subtlety, Calcification, Internal structure, Margin, Spiculation, Texture, Lobulation, Sphericity, Malignancy. Being able to learn to represent all of these characteristics of nodules would signify a good learnt representation.

#### Extraction

- We had scans were in a dicom format. We have firstly converted dicom to nifti and then converted nifti to nrrd format. The nrrd files have been converted to 3-d tensors.
- We have calculated the max of the diameters of the nodules and hence extracted a box of dimensions 64x64x64 from the CT scans. 
    
    - This step was necessary before training as the sizes of nodules compared to the size of lungs was very small and hence there would have been a lot of noise in the dataset for training.
    - Also each CT scan had nx512x512 slices. Where n varied for each CT scan and hence it was difficult to extract nodules and send through a model which accepts a fixed size of input.

However on extracting nodules from these scans we encountered a few difficulties due to the format of the dataset. 
Total number of annotations are 6859. Although the number of nodules might be less however all the annotations have been used during training.

After extraction we observed a few anomalies and this is how we  have proceeded.

- Each nodule has multiple annotations for each Nodule. In the format IL- IM etc for a single nodule. We have now considered all the annotations for each nodule for now in training. 
- As mentioned above the extracted nodules are of cubes of sizes 64x64x64. Although for some scans the nodule size  mx512x512 had m>n. These nodules have been skipped.
- We are yet to correct a glitch on our side. If the distance from the center to the edge is less than 32 we have skipped the nodule for now.
- 8 patients had two scans with different annotations and nodule numbers. Namely the patient id’s 132, 151, 315, 332, 355, 365, 442, 484. 
- Also during extraction we found that 2 patients namely the patient-id’s 238 and 585 have no nodules and all the characteristic fields were marked 0 and hence these two patients have been excluded. Reducing the total patient count from 1012 to 1010.
- Patient id 777 has 127 annotations.
 
Total number of patients with nodules>3mm are  875 and the total number of annotations for nodules>3mm are 6859.

- The remaing patients:

28
32
62
71
100
143
174
189
197
205
214
218
224
225
226
238
239
253
261
279
293
295
306
307
316
322
327
330
331
333
336
342
349
361
364
382
383
389
391
401
410
417
418
422
425
428
441
446
455
465
472
482
506
511
512
513
514
519
528
531
536
540
544
548
561
564
573
585
589
600
603
612
616
622
623
627
632
646
653
665
667
668
679
683
685
689
690
691
710
711
716
718
731
737
738
745
746
755
760
764
774
784
804
808
839
853
862
876
877
878
881
885
887
889
891
897
900
901
903
918
927
930
931
934
937
948
952
954
960
964
967
970
975
979
988
992
995


#### Labels

The labels have been extracted from the xml files associated with each patient from the associated xmls we have created a csv which stores patient\_id nodule\_id and all the 9 characteristics for 6859 annotations of nodules> 3mm. This csv is further converted to a tensor of size 6859x9 which is named all_labels.pt which is passed to the data loaders.


In [ ]:
import numpy as np
import os
import datasets as mt_datasets
import transforms as mt_transforms

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn

from collections import defaultdict
from pathlib import Path
import imageio

from utils import find

import random
import torch
import numpy as np
from sklearn.metrics import roc_auc_score
import resnet
from torchvision import transforms
import torchvision
from torch.utils.tensorboard import SummaryWriter
import datasets as mt_datasets
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
import imageio

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support
from sklearn.metrics import average_precision_score


In [ ]:
ROOT_DIR = "/scratch/dj1311/Lung_cancer/LIDC_conversion5/"

img_list = []
label_list = []
my_list = []


In [ ]:
for idx, (dirpath, dirnames, filenames) in enumerate(os.walk(ROOT_DIR)):
    if not dirnames:
        # for 1 patient, all files
        key = None
        values = []

        for f in filenames:            
            if f.endswith(".nii.gz"):
                if f.startswith("LIDC-IDRI"):
                    key = f
                else:
                    values.append(f)
            
        for value in values:
            img_list.append(os.path.join(dirpath, key))
            label_list.append(os.path.join(dirpath, value))
            nodule_id = value.split(".nrrd.nii.gz")[0].split("Annotation")[1]
            patient_id = key.split("LIDC-IDRI-")[1][0:4]
            my_list.append((patient_id, nodule_id))


img_list, label_list = img_list[6000:], label_list[6000:]
# print(img_list, label_list)
filename_pairs = list(zip(img_list, label_list))
train_dataset = mt_datasets.MRI2DSegmentationDataset(filename_pairs, transform=mt_transforms.ToTensor())



In [ ]:
C = 32
nodules = []
all_labels = []

f = open('dev', 'w')

for i, patient in enumerate(train_dataset):
    print(f"Processing patient {i}.")

    if patient is None:
        print(f"Skipping {i}:")
        continue
    gt = patient['gt']
    inp = patient['input']
    labels = find(*my_list[i])
    if not labels:
        print(f"Skipping {i}: { patient['filename']}, {patient['gt_filename']}, without lables.")
        continue

    depth = inp.shape[0]

    idx = gt.nonzero()
    min_, max_ =  idx.min(dim=0)[0], idx.max(dim=0)[0]
    start, end = (min_ +  max_)/2 - C, (min_ +  max_)/2 + C
    
    s_x, s_y, s_z = start.tolist()
    e_x, e_y, e_z = end.tolist()

    if s_x < 0:
        e_x += abs(s_x)
        s_x = 0

    if e_x > depth:
        s_x -= (e_x - depth)
        e_x = depth


    if s_y < 0:
        e_y += abs(s_y)
        s_y = 0

    if e_y > 512:
        s_y -= (e_y - depth)
        e_y = depth


    if s_z < 0:
        e_z += abs(s_z)
        s_z = 0

    if e_z > 512:
        s_z -= (e_z - depth)
        e_z = depth


    nodule = (gt * inp)[s_x:e_x, s_y:e_y, s_z:e_z]
    print(i, nodule.shape, patient['filename'], patient['gt_filename'], file=f)
    f.flush()
    if nodule.shape[0]== 64 and nodule.shape[1]== 64 and nodule.shape[2]== 64:
        nodules.append(nodule.unsqueeze(0))
        all_labels.append(torch.tensor(labels).unsqueeze(0))
    else:
        print(i,nodule.shape)


In [ ]:
nodules = torch.cat(nodules)
all_labels = torch.cat(all_labels, dim=0)

print(nodules.shape, all_labels.shape)

torch.save(nodules, 'nodules6000:.pt')
torch.save(all_labels, 'all_labels6000:.pt')


#### Data loaders

Designing the Data loader:- using the medical torch data loader as a guideline we create a data loader to load the tensors(after converting the nrrd to tensors). This data loader creates a pair of labels and the tensor of the nodule.A class named Data is created which returns a pair of tensor and a label corresponding to the particular nodule(all_labels.pt) whereas the tensor is loaded from nodule.pt

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

writer = SummaryWriter()

nodules = torch.load('concat_nodules.pt')
all_lables = torch.load('concat_lables.pt')

batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 42


def rotation(img):
    i, j = random.randint(0, 2), random.randint(0, 2)
    return img.transpose(i, j)


# In[4]:


dataset = mt_datasets.TensorDataset(nodules, all_lables)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


#### Model

The model used is a Resnet10. The last fully connected layer has been fine tuned to give outputs for multi class prediction for 9 features. 

The model is a multi class-multi label classifier.

In [ ]:
# resnet18 = models.resnet18(pretrained=True)
model = resnet.resnet18(sample_size=64, sample_duration=64, num_classes=[5])
# num_ftrs = resnet18.fc.in_features
# resnet18.fc = nn.Linear(num_ftrs, 5)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


print("="*20)
print('Training Started')
print("="*20)


epochs = 100
idx = 0
y_true = []
y_scores = []
for epoch in range(epochs):
    for i, (nodule, labels) in enumerate(train_loader):
        nodule, labels = nodule[0].to(device), labels.to(device)


        # get the inputs; data is a list of [inputs, labels]
        nodule = nodule.unsqueeze(1)
        nodule = torch.cat([nodule,nodule,nodule],dim=1)
        labels = labels[:, -1] - 1
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(nodule)       
      
        loss = 0.0
        for i, output in enumerate(outputs):
            loss += criterion(output, labels)
            
        loss.backward()
        optimizer.step()

        # print statistics
        writer.add_scalar('Loss/train', loss.item(), idx)
        idx += 1

    print(f"Epochs: {epoch}, i = {i}, Loss = {loss.item()}")
    

print('Finished Training')



#### Analysing the data

The mean across all 9 features is:-

    -subtlety              3.823787
    -internalStructure     1.013992
    -calcification         5.655152
    -sphericity            3.783997
    -margin                3.940825
    -lobulation            1.699461
    -spiculation           1.590439
    -texture               4.419910
    -malignancy            2.813147

#### Count for each feature

    Malignancy:- Counter({5: 691, 4: 962, 3: 2606, 2: 1580, 1: 1020, 0: 2})
    Texture:- Counter({5: 5020, 4: 823, 2: 155, 1: 480, 3: 381, 0: 2})
    Lobulation:-   Counter({3: 714, 5: 191, 1: 4117, 2: 1451, 4: 386, 0: 2})
    Spiculation:- Counter({3: 517, 5: 253, 4: 269, 1: 4620, 2: 1200, 0: 2})
    Margin:- Counter({2: 620, 4: 2149, 3: 896, 1: 364, 5: 2830, 0: 2})
    Sphericity:- Counter({3: 2096, 4: 2316, 5: 1845, 2: 583, 1: 19, 0: 2})
    Calcification:- Counter({6: 6017, 3: 709, 5: 75, 4: 42, 2: 12, 1: 4, 0: 2})
    InternalStructure:- Counter({1: 6819, 2: 11, 4: 27, 5: 1, 3: 1, 0: 2})
    Subtlety:- Counter({5: 2542, 1: 349, 2: 625, 4: 1897, 3: 1446, 0: 2})

The data seems skewed as two patients have empty nodule annotations having values 0 for all the features. 



#### Calculating PRECISION RECALL 

Curves

These curves help establish a trade off between true positive rate and the positive predictive value. These values are appropriate for imbalanced datasets.

#### ROC curves

Trade off between true positive rate and false positive rate.

#### Area Under the curve
The area under the curve is used as a summary for the model skill. A skillful model will assign  a higher probability to a randomly chosen real positive occurrence than a negative occurrence on average. SKillful models tend to blow up on top left of the plot.


In [ ]:
print("="*20)
print('Precision/Recall Calculation')
print("="*20)


y_test, y_scores = [], []
for i, (nodule, labels) in enumerate(validation_loader):
    nodule, labels = nodule[0].to(device), labels.to(device)

    # get the inputs; data is a list of [inputs, labels]
    nodule = nodule.unsqueeze(1)
    nodule = torch.cat([nodule,nodule,nodule],dim=1)
    labels = labels[:, -1] - 1

    # forward
    outputs = model(nodule)[0]     

    y_test.append(labels.detach().cpu())
    y_scores.append(outputs.detach().cpu())


y_test = torch.cat(y_test).numpy()
y_scores = torch.cat(y_scores).numpy()


y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
print(y_test.shape, y_scores.shape)
# precision, recall, _, _ = precision_recall_fscore_support(y_test, y_scores)
# print(precision, recall)




# Compute Precision-Recall and plot curve
precision = dict()
recall = dict()
average_precision = dict()
n_classes = 5
for i in range(n_classes):
#     precision[i], recall[i], _, _ = precision_recall_fscore_support(y_test[:, i], y_scores[:, i])
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_scores[:, i])
    # average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])
    print(precision[i][300], recall[i][300])


#### Results for precesion recall values for malignancy
    
         Precesion          Recall
    -0:- 0.3333333333333333 0.8
    -1:- 0.2925170068027211 0.6099290780141844
    -2:- 0.4391891891891892 0.5909090909090909
    -3:- 0.2033898305084746 0.759493670886076
    -4:- 0.19387755102040816 0.8769230769230769
    
    Average precision score, micro-averaged over all classes: 0.41
    
#### Results for Area under the curve calculation

The noted values after 100 epochs is:-

Epoch: 99, Accuracy: 40.59405940594059

For the malignancy label the value of area under the curve is:

    -0: 0.8182831661092531
    -1: 0.5827922077922078 
    -2: 0.5278716216216217
    -3: 0.45017482517482516
    -4: 0.6417525773195877}
